<a href="https://colab.research.google.com/github/tamucc-gcl/wrkshp_edna_metabarcoding/blob/main/analysis/colab_renv_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# R in Colab with renv Persisted on Google Drive

This notebook will:
1. Mount Google Drive and clone your GitHub project.
2. Restore your `renv` environment into a Drive-backed library so package installs persist across Colab sessions.
3. Run example R code to verify setup.


In [1]:
#Set-up Computing Environment 1

## Install Linux programs needed
#!apt install libglpk-dev pandoc

## Mount Google Drive
from google.colab import drive
import os
drive.mount('/content/drive')
os.environ['COLAB'] = 'TRUE'

## Clone GitHub Repo and move into that repo
local_path = '/content/edna_workshop'
repo  = "tamucc-gcl/wrkshp_edna_metabarcoding"
url = f"https://github.com/{repo}.git"
!git clone {url} {local_path}

## Setup R and move to local directory
os.chdir(local_path)
%reload_ext rpy2.ipython

Mounted at /content/drive
Cloning into '/content/edna_workshop'...
remote: Enumerating objects: 240, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 240 (delta 85), reused 170 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (240/240), 15.91 MiB | 18.68 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [2]:
%%R -i local_path
# Point R at your Drive-backed library
drive_lib <- "/content/drive/MyDrive/edna_libraries"
dir.create(drive_lib, recursive = TRUE, showWarnings = FALSE)
.libPaths(c(drive_lib, .libPaths()))

if (!require("renv", lib.loc = drive_lib, quietly = TRUE, warn.conflicts = FALSE)) {
  install.packages("renv", lib = drive_lib,
                   quietly = TRUE, warn.conflicts = FALSE)
}

trying URL 'https://packagemanager.posit.co/cran/__linux__/jammy/latest/src/contrib/renv_1.1.4.tar.gz'
Content type 'binary/octet-stream' length 1272549 bytes (1.2 MB)
downloaded 1.2 MB


The downloaded source packages are in
	‘/tmp/RtmpEQYdCj/downloaded_packages’
In addition: Warning message:
In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  library ‘/content/drive/MyDrive/edna_libraries’ contains no packages


In [3]:
%%R -i local_path
renv::restore(library = drive_lib,
              lockfile = paste0(local_path, '/renv.lock'),
              prompt = FALSE,
              exclude = c('ShortRead', 'SummarizedExperiment'))

The following package(s) will be updated:

# CRAN -----------------------------------------------------------------------
- renv   [repo: RSPM -> CRAN; ver: 1.1.4 -> 1.1.2]

# Downloading packages -------------------------------------------------------
- Downloading renv from CRAN ...                OK [1.2 Mb in 0.56s]
Successfully downloaded 1 package in 2.2 seconds.

# Installing packages --------------------------------------------------------
- Installing renv ...                           OK [built from source and cached in 19s]


In addition: There were 16 warnings (use warnings() to see them)


In [ ]:
%%R
library(vegan)
data(dune)
sol <- metaMDS(dune)
sol
plot(sol, type="t")

Error in library(vegan) : there is no package called ‘vegan’


RInterpreterError: Failed to parse and evaluate line 'library(vegan)\ndata(dune)\nsol <- metaMDS(dune)\nsol\nplot(sol, type="t")\n'.
R error message: 'Error in library(vegan) : there is no package called ‘vegan’'

Below is extra troubleshooting code

In [ ]:
drive.flush_and_unmount()

In [ ]:
import shutil
shutil.rmtree('/content/edna_workshop')


OSError: [Errno 107] Transport endpoint is not connected: '/content/edna_workshop'